In [1]:
import numpy as np
import collections.abc
collections.Iterable = collections.abc.Iterable
from collections import OrderedDict
import pylogit as pl
import pandas as pd
import scipy.special
import json

In [2]:
df = pd.read_csv('history.csv')
df

,round,arena,pirate1,pirate2,pirate3,pirate4,fa1,fa2,fa3,fa4,...,nfa4,opening_odds1,opening_odds2,opening_odds3,opening_odds4,closing_odds1,closing_odds2,closing_odds3,closing_odds4,winner
0,3574,0,7,19,4,13,0,0,0,0,...,0,5,2,3,9,6,2,3,11,3
1,3574,1,18,12,17,14,0,0,0,0,...,0,13,2,3,3,13,2,3,3,4
2,3574,2,2,16,9,3,0,0,0,0,...,0,13,2,5,4,13,2,6,5,2
3,3574,3,10,20,1,6,0,0,0,0,...,0,13,10,2,13,13,13,2,13,3
4,3574,4,8,15,11,5,0,0,0,0,...,0,13,2,11,6,13,2,13,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17450,8763,0,1,15,14,11,4,2,0,0,...,-2,3,2,13,13,4,2,13,13,2
17451,8763,1,9,3,8,20,-1,3,2,0,...,-1,13,7,6,2,13,9,9,2,2
17452,8763,2,2,12,6,19,1,0,1,-2,...,-3,10,8,4,2,13,10,5,2,4
17453,8763,3,10,7,13,18,0,3,0,-2,...,-3,13,2,3,9,13,2,3,11,2


In [3]:
df.describe()

,round,arena,pirate1,pirate2,pirate3,pirate4,fa1,fa2,fa3,fa4,...,nfa4,opening_odds1,opening_odds2,opening_odds3,opening_odds4,closing_odds1,closing_odds2,closing_odds3,closing_odds4,winner
count,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,...,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000
mean,6895.217416,2.000000,10.594958,10.437353,10.527757,10.439931,0.970496,0.988026,0.969350,0.983157,...,-1.209625,7.924148,7.042051,5.981438,4.687310,8.248525,7.416786,6.361673,5.057405,2.747522
std,1124.586097,1.414254,5.748542,5.784008,5.782101,5.749531,1.732478,1.735760,1.733119,1.732663,...,0.970576,4.460331,4.421288,4.184334,3.634178,4.532492,4.556024,4.384231,3.926012,1.104911
min,3574.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-5.000000,-5.000000,-5.000000,-5.000000,...,-6.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000
25%,5962.000000,1.000000,6.000000,5.000000,5.000000,5.000000,0.000000,0.000000,0.000000,0.000000,...,-2.000000,3.000000,3.000000,2.000000,2.000000,3.000000,3.000000,2.000000,2.000000,2.000000
50%,6918.000000,2.000000,11.000000,10.000000,11.000000,10.000000,1.000000,1.000000,1.000000,1.000000,...,-1.000000,7.000000,6.000000,4.000000,3.000000,9.000000,7.000000,5.000000,3.000000,3.000000
75%,7891.000000,3.000000,16.000000,15.000000,16.000000,15.000000,2.000000,2.000000,2.000000,2.000000,...,0.000000,13.000000,13.000000,10.000000,6.000000,13.000000,13.000000,11.000000,7.000000,4.000000
max,8763.000000,4.000000,20.000000,20.000000,20.000000,20.000000,8.000000,8.000000,8.000000,9.000000,...,0.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,4.000000


In [4]:
# combine columns pirate1 through pirate4 into a single column containing a list of pirate choices
def convert_df(df_orig):
    df = df_orig.copy()
    df['pirates'] = df[['pirate1', 'pirate2', 'pirate3', 'pirate4']].values.tolist()
    df['fas'] = df[['fa1', 'fa2', 'fa3', 'fa4']].values.tolist()
    df['pfas'] = df[['pfa1', 'pfa2', 'pfa3', 'pfa4']].values.tolist()
    df['nfas'] = df[['nfa1', 'nfa2', 'nfa3', 'nfa4']].values.tolist()
    df['opening_odds'] = df[['opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4']].values.tolist()
    df['closing_odds'] = df[['closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']].values.tolist()
    df = df.drop(['pirate1', 'pirate2', 'pirate3', 'pirate4'], axis=1)
    df = df.drop(['fa1', 'fa2', 'fa3', 'fa4'], axis=1)
    df = df.drop(['pfa1', 'pfa2', 'pfa3', 'pfa4'], axis=1)
    df = df.drop(['nfa1', 'nfa2', 'nfa3', 'nfa4'], axis=1)
    df = df.drop(['opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4'], axis=1)
    df = df.drop(['closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4'], axis=1)
    df['match_id'] = df['round'] * 5 + df['arena']
    return df
df = convert_df(df)
df

,round,arena,winner,pirates,fas,pfas,nfas,opening_odds,closing_odds,match_id
0,3574,0,3,"[7, 19, 4, 13]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[5, 2, 3, 9]","[6, 2, 3, 11]",17870
1,3574,1,4,"[18, 12, 17, 14]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 3, 3]","[13, 2, 3, 3]",17871
2,3574,2,2,"[2, 16, 9, 3]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 5, 4]","[13, 2, 6, 5]",17872
3,3574,3,3,"[10, 20, 1, 6]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 10, 2, 13]","[13, 13, 2, 13]",17873
4,3574,4,2,"[8, 15, 11, 5]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 11, 6]","[13, 2, 13, 6]",17874
...,...,...,...,...,...,...,...,...,...,...
17450,8763,0,2,"[1, 15, 14, 11]","[4, 2, 0, 0]","[5, 3, 2, 2]","[-1, -1, -2, -2]","[3, 2, 13, 13]","[4, 2, 13, 13]",43815
17451,8763,1,2,"[9, 3, 8, 20]","[-1, 3, 2, 0]","[0, 4, 4, 1]","[-1, -1, -2, -1]","[13, 7, 6, 2]","[13, 9, 9, 2]",43816
17452,8763,2,4,"[2, 12, 6, 19]","[1, 0, 1, -2]","[3, 0, 2, 1]","[-2, 0, -1, -3]","[10, 8, 4, 2]","[13, 10, 5, 2]",43817
17453,8763,3,2,"[10, 7, 13, 18]","[0, 3, 0, -2]","[1, 3, 3, 1]","[-1, 0, -3, -3]","[13, 2, 3, 9]","[13, 2, 3, 11]",43818


In [5]:
# explode each row of df into four rows, so that the resulting df
# contains columns of round, arena, win (0 or 1), pirate, fa, opening_odds, closing_odds, match_id
def convert_to_long_format_with_win(df):
    long_format_columns = ['round', 'arena', 'win', 'pirate', 'fa', 'pfa', 'nfa', 'position', 'position_factor', 'is_pos1', 'is_pos2', 'is_pos3', 'is_pos4', 'opening_odds', 'closing_odds', 'match_id']
    # positional effect is significant but sorta non-linear; this is taken from a model data
    # and we'll calculate each pirates "suscetibility to positional effects" by fitting a multiplier
    position_factor = [-0.1821, 0, 0.2814, 0.5523]
    long_format = pd.DataFrame([
        (
            tup.round,
            tup.arena,
            1 if 'winner' in df.columns and tup.winner == (i + 1) else 0,
            tup.pirates[i],
            tup.fas[i],
            tup.pfas[i],
            tup.nfas[i],
            i + 1,
            position_factor[i],
            1 if i + 1 == 1 else 0,
            1 if i + 1 == 2 else 0,
            1 if i + 1 == 3 else 0,
            1 if i + 1 == 4 else 0,
            tup.opening_odds[i],
            tup.closing_odds[i],
            tup.match_id
        )
        for tup in df.itertuples() for i in range(4)
    ],columns=long_format_columns)
    long_format['log_opening_implied_winrate'] = np.log(1.0 / long_format['opening_odds'])
    return long_format

def convert_to_long_format_without_win(df):
    return convert_to_long_format_with_win(df).drop(['win'], axis=1)

long_format = convert_to_long_format_with_win(df)
long_format

,round,arena,win,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,3574,0,0,7,0,0,0,1,-0.1821,1,0,0,0,5,6,17870,-1.609438
1,3574,0,0,19,0,0,0,2,0.0000,0,1,0,0,2,2,17870,-0.693147
2,3574,0,1,4,0,0,0,3,0.2814,0,0,1,0,3,3,17870,-1.098612
3,3574,0,0,13,0,0,0,4,0.5523,0,0,0,1,9,11,17870,-2.197225
4,3574,1,0,18,0,0,0,1,-0.1821,1,0,0,0,13,13,17871,-2.564949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69815,8763,3,0,18,-2,1,-3,4,0.5523,0,0,0,1,9,11,43818,-2.197225
69816,8763,4,0,17,5,6,-1,1,-0.1821,1,0,0,0,2,2,43819,-0.693147
69817,8763,4,0,5,1,1,0,2,0.0000,0,1,0,0,5,5,43819,-1.609438
69818,8763,4,0,16,1,2,-1,3,0.2814,0,0,1,0,4,4,43819,-1.386294


In [6]:
PIRATE_NAMES = {
    1: "Dan",
    2: "Sproggie",
    3: "Orvinn",
    4: "Lucky",
    5: "Edmund",
    6: "Peg Leg",
    7: "Bonnie",
    8: "Puffo",
    9: "Stuff",
    10: "Squire",
    11: "Crossblades",
    12: "Stripey",
    13: "Ned",
    14: "Fairfax",
    15: "Gooblah",
    16: "Franchisco",
    17: "Federismo",
    18: "Blackbeard",
    19: "Buck",
    20: "Tailhook",
}

In [7]:
pirates_including_the_first = list(range(1, 21))
pirates_except_the_first = list(range(2, 21))

In [24]:
with_loiw_specification = OrderedDict()
with_loiw_names = OrderedDict()
with_loiw_specification["intercept"] = pirates_except_the_first
with_loiw_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
# we use the same FA coefficient and same opening implied winrate coefficient for all pirates
with_loiw_specification["fa"] = [pirates_including_the_first]
with_loiw_names["fa"] = ['FA']
with_loiw_specification["log_opening_implied_winrate"] = [pirates_including_the_first]
with_loiw_names["log_opening_implied_winrate"] = ['log_opening_implied_winrate']
with_loiw_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=with_loiw_specification,
model_type='MNL',
names=with_loiw_names)
with_loiw_mnl.fit_mle(np.zeros(21))
with_loiw_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.24 seconds.
Final log-likelihood: -18,235.0666


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,824
Method:                                MLE   Df Model:                           21
Date:                     Mon, 09 Jan 2023   Pseudo R-squ.:                   0.219
Time:                             00:19:37   Pseudo R-bar-squ.:               0.218
AIC:                            36,512.133   Log-Likelihood:            -18,235.067
BIC:                            36,674.501   LL-Null:                   -23,352.129
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
ASC_2_Sproggie                 -0.3015      0.064     -4.747      0.000      -0.426      -0.177
ASC_3_Orvinn                   -0.6093      0.082     -7.432      0.000      -0.770      -0.449
ASC_4_Lucky                    -0.1268      0.055     -2.319      0.020      -0.234      -0.020
ASC_5_Edmund                   -0.1442      0.055     -2.600      0.009      -0.253      -0.036
ASC_6_Peg Leg                  -0.3446      0.068     -5.051      0.000      -0.478      -0.211
ASC_7_Bonnie                   -0.3013      0.059     -5.084      0.000      -0.418      -0.185
ASC_8_Puffo                    -0.4196      0.074     -5.696      0.000      -0.564      -0.275
ASC_9_Stuff                    -0.7360      0.094     -7.799      0.000      -0.921      -0.551
ASC_10_Squire                  -0.6544      0.090     -7.307      0.000      -0.830      -0.479
ASC_11_Crossblades             -0.5264      0.079     -6.680      0.000      -0.681      -0.372
ASC_12_Stripey                 -0.2692      0.062     -4.333      0.000      -0.391      -0.147
ASC_13_Ned                     -0.1619      0.058     -2.806      0.005      -0.275      -0.049
ASC_14_Fairfax                 -0.4022      0.065     -6.163      0.000      -0.530      -0.274
ASC_15_Gooblah                  0.4210      0.053      7.962      0.000       0.317       0.525
ASC_16_Franchisco              -0.1499      0.052     -2.875      0.004      -0.252      -0.048
ASC_17_Federismo               -0.1096      0.052     -2.103      0.035      -0.212      -0.007
ASC_18_Blackbeard              -0.3519      0.066     -5.325      0.000      -0.481      -0.222
ASC_19_Buck                     0.2487      0.052      4.751      0.000       0.146       0.351
ASC_20_Tailhook                -0.0842      0.053     -1.584      0.113      -0.188       0.020
FA                              0.0951      0.009     11.029      0.000       0.078       0.112
log_opening_implied_winrate     0.8761      0.028     31.458      0.000       0.821       0.931
===============================================================================================
"""

In [25]:
# train a even more sophisticated model that includes the position of the pirate in the lineup
with_loiw_and_position_specification = OrderedDict()
with_loiw_and_position_names = OrderedDict()
with_loiw_and_position_specification["intercept"] = pirates_except_the_first
with_loiw_and_position_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
with_loiw_and_position_specification["fa"] = [pirates_including_the_first]
with_loiw_and_position_names["fa"] = ['FA']
with_loiw_and_position_specification["log_opening_implied_winrate"] = [pirates_including_the_first]
with_loiw_and_position_names["log_opening_implied_winrate"] = ['log_opening_implied_winrate']
with_loiw_and_position_specification["position"] = [pirates_including_the_first]
with_loiw_and_position_names["position"] = ['position']
with_loiw_and_position_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=with_loiw_and_position_specification,
model_type='MNL',
names=with_loiw_and_position_names)
with_loiw_and_position_mnl.fit_mle(np.zeros(22))
with_loiw_and_position_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.34 seconds.
Final log-likelihood: -18,145.0088


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,823
Method:                                MLE   Df Model:                           22
Date:                     Mon, 09 Jan 2023   Pseudo R-squ.:                   0.223
Time:                             00:19:40   Pseudo R-bar-squ.:               0.222
AIC:                            36,334.018   Log-Likelihood:            -18,145.009
BIC:                            36,504.117   LL-Null:                   -23,352.129
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
ASC_2_Sproggie                 -0.6864      0.070     -9.779      0.000      -0.824      -0.549
ASC_3_Orvinn                   -1.2715      0.097    -13.109      0.000      -1.462      -1.081
ASC_4_Lucky                    -0.2781      0.056     -4.950      0.000      -0.388      -0.168
ASC_5_Edmund                   -0.3329      0.058     -5.780      0.000      -0.446      -0.220
ASC_6_Peg Leg                  -0.7212      0.074     -9.726      0.000      -0.866      -0.576
ASC_7_Bonnie                   -0.5968      0.064     -9.373      0.000      -0.722      -0.472
ASC_8_Puffo                    -0.9639      0.085    -11.317      0.000      -1.131      -0.797
ASC_9_Stuff                    -1.3771      0.107    -12.833      0.000      -1.587      -1.167
ASC_10_Squire                  -1.2387      0.101    -12.295      0.000      -1.436      -1.041
ASC_11_Crossblades             -1.1123      0.091    -12.208      0.000      -1.291      -0.934
ASC_12_Stripey                 -0.6555      0.069     -9.495      0.000      -0.791      -0.520
ASC_13_Ned                     -0.3668      0.060     -6.117      0.000      -0.484      -0.249
ASC_14_Fairfax                 -0.8174      0.073    -11.221      0.000      -0.960      -0.675
ASC_15_Gooblah                  0.6573      0.056     11.635      0.000       0.547       0.768
ASC_16_Franchisco              -0.2924      0.054     -5.449      0.000      -0.398      -0.187
ASC_17_Federismo               -0.2564      0.054     -4.768      0.000      -0.362      -0.151
ASC_18_Blackbeard              -0.7572      0.073    -10.331      0.000      -0.901      -0.614
ASC_19_Buck                     0.3713      0.054      6.919      0.000       0.266       0.477
ASC_20_Tailhook                -0.1907      0.054     -3.526      0.000      -0.297      -0.085
FA                              0.1730      0.011     16.307      0.000       0.152       0.194
log_opening_implied_winrate     0.5182      0.039     13.415      0.000       0.442       0.594
position                        0.1458      0.011     13.288      0.000       0.124       0.167
===============================================================================================
"""

In [26]:
# train a reduced model without log opening implied winrate
# i.e. we assume that the opening odds don't tell us any extra information
simple_specification = OrderedDict()
simple_names = OrderedDict()
simple_specification["intercept"] = pirates_except_the_first
simple_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
# we use the same FA coefficient and same opening implied winrate coefficient for all pirates
simple_specification["fa"] = [pirates_including_the_first]
simple_names["fa"] = ['FA']

simple_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=simple_specification,
model_type='MNL',
names=simple_names)
simple_mnl.fit_mle(np.zeros(20))
simple_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.41 seconds.
Final log-likelihood: -18,750.3869


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,825
Method:                                MLE   Df Model:                           20
Date:                     Mon, 09 Jan 2023   Pseudo R-squ.:                   0.197
Time:                             00:19:42   Pseudo R-bar-squ.:               0.196
AIC:                            37,540.774   Log-Likelihood:            -18,750.387
BIC:                            37,695.410   LL-Null:                   -23,352.129
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
ASC_2_Sproggie        -1.1955      0.057    -21.044      0.000      -1.307      -1.084
ASC_3_Orvinn          -2.1749      0.067    -32.570      0.000      -2.306      -2.044
ASC_4_Lucky           -0.4858      0.053     -9.130      0.000      -0.590      -0.381
ASC_5_Edmund          -0.5922      0.053    -11.110      0.000      -0.697      -0.488
ASC_6_Peg Leg         -1.2232      0.062    -19.679      0.000      -1.345      -1.101
ASC_7_Bonnie          -0.9844      0.055    -17.846      0.000      -1.092      -0.876
ASC_8_Puffo           -1.6893      0.063    -26.957      0.000      -1.812      -1.567
ASC_9_Stuff           -2.2721      0.082    -27.691      0.000      -2.433      -2.111
ASC_10_Squire         -2.0243      0.079    -25.583      0.000      -2.179      -1.869
ASC_11_Crossblades    -1.8984      0.067    -28.489      0.000      -2.029      -1.768
ASC_12_Stripey        -1.1772      0.055    -21.348      0.000      -1.285      -1.069
ASC_13_Ned            -0.6512      0.055    -11.808      0.000      -0.759      -0.543
ASC_14_Fairfax        -1.3846      0.058    -23.989      0.000      -1.498      -1.271
ASC_15_Gooblah         0.9588      0.051     18.983      0.000       0.860       1.058
ASC_16_Franchisco     -0.4885      0.051     -9.593      0.000      -0.588      -0.389
ASC_17_Federismo      -0.4654      0.051     -9.112      0.000      -0.565      -0.365
ASC_18_Blackbeard     -1.3032      0.059    -22.103      0.000      -1.419      -1.188
ASC_19_Buck            0.5275      0.052     10.212      0.000       0.426       0.629
ASC_20_Tailhook       -0.3319      0.052     -6.349      0.000      -0.434      -0.229
FA                     0.2798      0.007     42.317      0.000       0.267       0.293
======================================================================================
"""

In [20]:
# train a reduced model without log opening implied winrate but includes position
# i.e. we assume that the different positions are rigged, the opening odds don't tell us any extra information
with_position_specification = OrderedDict()
with_position_names = OrderedDict()
with_position_specification["intercept"] = pirates_except_the_first
with_position_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
# we use the same FA coefficient and same opening implied winrate coefficient for all pirates
with_position_specification["fa"] = [pirates_including_the_first]
with_position_names["fa"] = ['FA']
with_position_specification["position"] = [pirates_including_the_first]
with_position_names["position"] = ['position']

with_position_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=with_position_specification,
model_type='MNL',
names=with_position_names)
with_position_mnl.fit_mle(np.zeros(21))
with_position_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.29 seconds.
Final log-likelihood: -18,235.2126


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,824
Method:                                MLE   Df Model:                           21
Date:                     Sun, 15 Jan 2023   Pseudo R-squ.:                   0.219
Time:                             01:42:57   Pseudo R-bar-squ.:               0.218
AIC:                            36,512.425   Log-Likelihood:            -18,235.213
BIC:                            36,674.793   LL-Null:                   -23,352.129
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
ASC_2_Sproggie        -1.2326      0.058    -21.391      0.000      -1.346      -1.120
ASC_3_Orvinn          -2.2261      0.068    -32.912      0.000      -2.359      -2.094
ASC_4_Lucky           -0.4951      0.054     -9.166      0.000      -0.601      -0.389
ASC_5_Edmund          -0.6035      0.054    -11.142      0.000      -0.710      -0.497
ASC_6_Peg Leg         -1.2516      0.063    -19.879      0.000      -1.375      -1.128
ASC_7_Bonnie          -1.0144      0.056    -18.140      0.000      -1.124      -0.905
ASC_8_Puffo           -1.7435      0.063    -27.459      0.000      -1.868      -1.619
ASC_9_Stuff           -2.3205      0.083    -28.039      0.000      -2.483      -2.158
ASC_10_Squire         -2.0831      0.080    -26.068      0.000      -2.240      -1.926
ASC_11_Crossblades    -1.9504      0.068    -28.892      0.000      -2.083      -1.818
ASC_12_Stripey        -1.2068      0.056    -21.546      0.000      -1.317      -1.097
ASC_13_Ned            -0.6591      0.056    -11.775      0.000      -0.769      -0.549
ASC_14_Fairfax        -1.4113      0.058    -24.126      0.000      -1.526      -1.297
ASC_15_Gooblah         0.9961      0.051     19.411      0.000       0.895       1.097
ASC_16_Franchisco     -0.4977      0.052     -9.613      0.000      -0.599      -0.396
ASC_17_Federismo      -0.4702      0.052     -9.064      0.000      -0.572      -0.368
ASC_18_Blackbeard     -1.3384      0.060    -22.384      0.000      -1.456      -1.221
ASC_19_Buck            0.5466      0.053     10.406      0.000       0.444       0.650
ASC_20_Tailhook       -0.3407      0.053     -6.415      0.000      -0.445      -0.237
FA                     0.2864      0.007     42.667      0.000       0.273       0.300
position               0.2504      0.008     31.589      0.000       0.235       0.266
======================================================================================
"""

In [8]:
POSITIVE_FAS = {
    1: {1: 2, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 2, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 2, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    2: {1: 1, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0},
    3: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 1, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 1},
    4: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 1},
    5: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    6: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    7: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    8: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 1, 38: 1, 39: 0, 40: 0},
    9: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 1},
    10: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    11: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 1},
    12: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0},
    13: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    14: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 1, 27: 1, 28: 0, 29: 1, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    15: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    16: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 2, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    17: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 2, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 2},
    18: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 1, 27: 1, 28: 0, 29: 1, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    19: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    20: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0}
}
NEGATIVE_FAS = {
    1: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    2: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 1},
    3: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    4: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 1},
    5: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    6: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    7: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    8: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    9: {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0},
    10: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    11: {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    12: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    13: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    14: {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    15: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0},
    16: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    17: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    18: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    19: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    20: {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0}
}

def get_ongoing_df_from_file(path):
    columns = ['round', 'arena', 'pirate1', 'pirate2', 'pirate3', 'pirate4', 'fa1', 'fa2', 'fa3', 'fa4', 'opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4', 'closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']
    df = pd.DataFrame([], columns=columns)
    with open(path) as json_file:
        data = json.load(json_file)
        round = data["round"]
        for arena in range(5):
            pirates = data["pirates"][arena]
            # older rounds don't have fa data
            fas = [0 for pirate in pirates]
            if "foods" in data:
                foods = data["foods"][arena]
                positive_fas = [sum(POSITIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                negative_fas = [-sum(NEGATIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                fas = [sum(x) for x in zip(positive_fas, negative_fas)]
            pirate1, pirate2, pirate3, pirate4 = pirates
            fa1, fa2, fa3, fa4 = fas
            _, opening_odds1, opening_odds2, opening_odds3, opening_odds4 = data["openingOdds"][arena]
            _, closing_odds1, closing_odds2, closing_odds3, closing_odds4 = data["currentOdds"][arena]
            df_temp = pd.DataFrame([[round, arena, pirate1, pirate2, pirate3, pirate4, fa1, fa2, fa3, fa4, opening_odds1, opening_odds2, opening_odds3, opening_odds4, closing_odds1, closing_odds2, closing_odds3, closing_odds4]], columns=columns)
            df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [9]:
# backtesting
def get_data_for_round(df, round):
    return df[df['round'] == round]

convert_to_long_format_with_win(get_data_for_round(df, 8641))


,round,arena,win,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,8641,0,0,19,2,2,0,1,-0.1821,1,0,0,0,4,4,43205,-1.386294
1,8641,0,0,12,4,5,-1,2,0.0000,0,1,0,0,13,13,43205,-2.564949
2,8641,0,0,2,3,5,-2,3,0.2814,0,0,1,0,13,13,43205,-2.564949
3,8641,0,1,15,3,4,-1,4,0.5523,0,0,0,1,2,2,43205,-0.693147
4,8641,1,0,10,1,1,0,1,-0.1821,1,0,0,0,13,13,43206,-2.564949
5,8641,1,0,20,-1,1,-2,2,0.0000,0,1,0,0,8,12,43206,-2.079442
6,8641,1,0,8,-2,1,-3,3,0.2814,0,0,1,0,13,13,43206,-2.564949
7,8641,1,1,4,3,3,0,4,0.5523,0,0,0,1,2,2,43206,-0.693147
8,8641,2,1,17,4,5,-1,1,-0.1821,1,0,0,0,2,2,43207,-0.693147
9,8641,2,0,7,-1,1,-2,2,0.0000,0,1,0,0,6,6,43207,-1.791759


In [10]:
def generate_bets():
    for i in range(5):
        for j in range(5):
            for k in range(5):
                for l in range(5):
                    for m in range(5):
                        if i != 0 or j != 0 or k != 0 or l != 0 or m != 0:
                            yield (i, j, k, l, m)

len(list(generate_bets()))

3124

In [11]:
def get_odds_of_bet(bet, odds):
    prod = 1
    for arena, pos in enumerate(bet):
        if pos != 0:
            prod *= odds[arena * 4 + pos - 1]
    return prod

tmp = convert_to_long_format_with_win(get_data_for_round(df, 8641))
get_odds_of_bet((4,2,0,0,2), tmp['closing_odds'])

96

In [35]:
tmp

,round,arena,win,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,8641,0,0,19,2,2,0,1,-0.1821,1,0,0,0,4,4,43205,-1.386294
1,8641,0,0,12,4,5,-1,2,0.0000,0,1,0,0,13,13,43205,-2.564949
2,8641,0,0,2,3,5,-2,3,0.2814,0,0,1,0,13,13,43205,-2.564949
3,8641,0,1,15,3,4,-1,4,0.5523,0,0,0,1,2,2,43205,-0.693147
4,8641,1,0,10,1,1,0,1,-0.1821,1,0,0,0,13,13,43206,-2.564949
5,8641,1,0,20,-1,1,-2,2,0.0000,0,1,0,0,8,12,43206,-2.079442
6,8641,1,0,8,-2,1,-3,3,0.2814,0,0,1,0,13,13,43206,-2.564949
7,8641,1,1,4,3,3,0,4,0.5523,0,0,0,1,2,2,43206,-0.693147
8,8641,2,1,17,4,5,-1,1,-0.1821,1,0,0,0,2,2,43207,-0.693147
9,8641,2,0,7,-1,1,-2,2,0.0000,0,1,0,0,6,6,43207,-1.791759


In [12]:
def get_estimated_probabilities(df, model):
    return model.predict(df)

# get_estimated_probabilities(tmp, with_position_mnl)

In [13]:
def get_estimated_probability_of_bet(bet, probabilities):
    prod = 1
    for arena, pos in enumerate(bet):
        if pos != 0:
            prod *= probabilities[arena * 4 + pos - 1]
    return prod

# get_estimated_probability_of_bet((4,2,0,0,2), get_estimated_probabilities(tmp, with_position_mnl))

In [14]:
def get_estimated_return_per_unit_of_bet(bet, odds, probabilities):
    return get_odds_of_bet(bet, odds) * get_estimated_probability_of_bet(bet, probabilities)

# get_estimated_return_per_unit_of_bet((4,2,0,0,2), tmp['closing_odds'], get_estimated_probabilities(tmp, with_position_mnl))

In [15]:
def get_max_possible_bet_amount_floor(odds):
    return 1000000 // odds

def get_max_possible_bet_amount_ceil(odds):
    return -(1000000 // -odds)

get_max_possible_bet_amount_floor(96)
get_max_possible_bet_amount_ceil(96)

10417

In [16]:
def get_estimated_return_of_bet_amount(bet, odds, probabilities, bet_amount):
    odds_of_bet = get_odds_of_bet(bet, odds)
    estimated_probability = get_estimated_probability_of_bet(bet, probabilities)
    payout = min(1000000, bet_amount * odds_of_bet)
    return payout * estimated_probability

def get_best_bet_amount_and_estimated_return(bet, odds, probabilities):
    max_possible_bet_amount_floor = get_max_possible_bet_amount_floor(get_odds_of_bet(bet, odds))
    max_possible_bet_amount_ceil = get_max_possible_bet_amount_ceil(get_odds_of_bet(bet, odds))
    estimated_return_bet_amount_floor = get_estimated_return_of_bet_amount(bet, odds, probabilities, max_possible_bet_amount_floor)
    estimated_return_bet_amount_ceil = get_estimated_return_of_bet_amount(bet, odds, probabilities, max_possible_bet_amount_ceil)
    if (estimated_return_bet_amount_floor - max_possible_bet_amount_floor) >= (estimated_return_bet_amount_ceil - max_possible_bet_amount_ceil):
        return max_possible_bet_amount_floor, estimated_return_bet_amount_floor
    else:
        return max_possible_bet_amount_ceil, estimated_return_bet_amount_ceil

def get_best_bet_amount_and_estimated_return_given_personal_cap(bet, odds, probabilities, cap):
    amount, estimated_return = get_best_bet_amount_and_estimated_return(bet, odds, probabilities)
    if cap >= amount:
        return amount, estimated_return
    else:
        return cap, get_estimated_return_of_bet_amount(bet, odds, probabilities, cap)


In [17]:
def get_profitable_bets(odds, probabilities, cap):
    for bet in generate_bets():
        if get_estimated_return_per_unit_of_bet(bet, odds, probabilities) > 1:
            bet_amount, estimated_return = get_best_bet_amount_and_estimated_return_given_personal_cap(bet, odds, probabilities, cap)
            estimated_profit = estimated_return - bet_amount
            return_if_won = min(1000000, bet_amount * get_odds_of_bet(bet, odds))
            yield bet, bet_amount, estimated_return, estimated_profit, return_if_won

# 10 best bets for round 8641
# sorted(list(get_profitable_bets(tmp['closing_odds'], get_estimated_probabilities(tmp, with_position_mnl), 11376)), key=lambda x: x[3], reverse=True)[:10]

In [18]:
def is_winning_bet(bet, winners):
    for arena, pos in enumerate(bet):
        if pos != 0 and winners[arena] != pos:
            return False
    return True

def get_actual_return_of_bet(bet, winners, return_if_won):
    if is_winning_bet(bet, winners):
        return min(1000000, return_if_won)
    else:
        return 0


In [19]:
history_rounds = df["round"].unique().tolist()
len(history_rounds)

3491

In [20]:
from tqdm import tqdm
def backtest_model_with_cap(mnl, cap, history_rounds):
    total_bet_amount = 0
    total_return = 0
    total_potential_bet_amount = cap * 10 * len(history_rounds)
    for round in tqdm(history_rounds):
        tmp = convert_to_long_format_with_win(get_data_for_round(df, round))
        winners = df[df['round'] == round]['winner'].tolist()
        best_ten_bets = sorted(list(get_profitable_bets(tmp['closing_odds'], get_estimated_probabilities(tmp, mnl), cap)), key=lambda x: x[3], reverse=True)[:10]
        for bet, bet_amount, estimated_return, estimated_profit, return_if_won in best_ten_bets:
            total_bet_amount += bet_amount
            total_return += get_actual_return_of_bet(bet, winners, return_if_won)
    return total_bet_amount, total_potential_bet_amount, total_return, total_return - total_bet_amount, 1 + (total_return - total_bet_amount) / total_potential_bet_amount

In [130]:
backtest_model_with_cap(with_position_mnl, 10000, history_rounds)

100%|██████████| 3369/3369 [03:11<00:00, 17.63it/s]


(325253969, 336900000, 647768680, 322514711, 1.9573010121697834)

In [131]:
backtest_model_with_cap(with_position_mnl, 1000, history_rounds)

100%|██████████| 3369/3369 [03:20<00:00, 16.83it/s]


(33355474, 33690000, 60393856, 27038382, 1.8025640249332147)

In [132]:
backtest_model_with_cap(with_position_mnl, 100, history_rounds)

100%|██████████| 3369/3369 [03:26<00:00, 16.34it/s]


(3362857, 3369000, 6586100, 3223243, 1.9567358266547936)

In [133]:
backtest_model_with_cap(with_position_mnl, 10, history_rounds)

100%|██████████| 3369/3369 [03:28<00:00, 16.18it/s]


(336868, 336900, 588040, 251172, 1.7455387355298309)

In [134]:
backtest_model_with_cap(with_position_mnl, 1, history_rounds)

100%|██████████| 3369/3369 [03:28<00:00, 16.14it/s]


(33689, 33690, 58804, 25115, 1.7454734342534877)

In [164]:
# next, we train a model where pirates share one coeff for favorites and one coeff for allergies
split_fa_specification = OrderedDict()
split_fa_names = OrderedDict()
split_fa_specification["intercept"] = pirates_except_the_first
split_fa_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
# we use the same FA coefficient and same opening implied winrate coefficient for all pirates
split_fa_specification["pfa"] = [pirates_including_the_first]
split_fa_names["pfa"] = ['PFA']
split_fa_specification["nfa"] = [pirates_including_the_first]
split_fa_names["nfa"] = ['NFA']
split_fa_specification["position"] = [pirates_including_the_first]
split_fa_names["position"] = ['position']

split_fa_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=split_fa_specification,
model_type='MNL',
names=split_fa_names)
split_fa_mnl.fit_mle(np.zeros(22))
split_fa_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.36 seconds.
Final log-likelihood: -18,142.6405


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,823
Method:                                MLE   Df Model:                           22
Date:                     Wed, 04 Jan 2023   Pseudo R-squ.:                   0.223
Time:                             18:59:28   Pseudo R-bar-squ.:               0.222
AIC:                            36,329.281   Log-Likelihood:            -18,142.641
BIC:                            36,499.381   LL-Null:                   -23,352.129
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
ASC_2_Sproggie        -1.1866      0.058    -20.518      0.000      -1.300      -1.073
ASC_3_Orvinn          -2.1865      0.068    -32.337      0.000      -2.319      -2.054
ASC_4_Lucky           -0.5921      0.055    -10.840      0.000      -0.699      -0.485
ASC_5_Edmund          -0.7312      0.055    -13.277      0.000      -0.839      -0.623
ASC_6_Peg Leg         -1.4406      0.065    -22.283      0.000      -1.567      -1.314
ASC_7_Bonnie          -1.1051      0.056    -19.577      0.000      -1.216      -0.994
ASC_8_Puffo           -1.6565      0.064    -25.919      0.000      -1.782      -1.531
ASC_9_Stuff           -2.4872      0.084    -29.661      0.000      -2.652      -2.323
ASC_10_Squire         -2.2411      0.081    -27.696      0.000      -2.400      -2.083
ASC_11_Crossblades    -2.0420      0.068    -30.065      0.000      -2.175      -1.909
ASC_12_Stripey        -1.2472      0.056    -22.210      0.000      -1.357      -1.137
ASC_13_Ned            -0.6811      0.056    -12.134      0.000      -0.791      -0.571
ASC_14_Fairfax        -1.4607      0.059    -24.900      0.000      -1.576      -1.346
ASC_15_Gooblah         0.9023      0.052     17.379      0.000       0.801       1.004
ASC_16_Franchisco     -0.5020      0.052     -9.685      0.000      -0.604      -0.400
ASC_17_Federismo      -0.5001      0.052     -9.645      0.000      -0.602      -0.398
ASC_18_Blackbeard     -1.2987      0.060    -21.677      0.000      -1.416      -1.181
ASC_19_Buck            0.4531      0.053      8.526      0.000       0.349       0.557
ASC_20_Tailhook       -0.4847      0.054     -8.922      0.000      -0.591      -0.378
PFA                    0.2134      0.009     24.921      0.000       0.197       0.230
NFA                    0.4185      0.012     34.918      0.000       0.395       0.442
position               0.2510      0.008     31.597      0.000       0.235       0.267
======================================================================================
"""

In [146]:
backtest_model_with_cap(split_fa_mnl, 1, history_rounds)

100%|██████████| 3369/3369 [03:25<00:00, 16.41it/s]


(33682, 33690, 96461, 62779, 2.863431285247848)

In [147]:
backtest_model_with_cap(split_fa_mnl, 10000, history_rounds)

100%|██████████| 3369/3369 [03:07<00:00, 17.99it/s]


(326782178, 336900000, 649728421, 322946243, 1.9585819026417335)

In [ ]:
# OMFG splitting favorite count and allergy count is absolutely necessary!

In [25]:
# next step is to split the coefficients for positional (dis-)advantages
split_position_specification = OrderedDict()
split_position_names = OrderedDict()
split_position_specification["intercept"] = pirates_except_the_first
split_position_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
# we use the same FA coefficient and same opening implied winrate coefficient for all pirates
split_position_specification["pfa"] = [pirates_including_the_first]
split_position_names["pfa"] = ['PFA']
split_position_specification["nfa"] = [pirates_including_the_first]
split_position_names["nfa"] = ['NFA']
# split_position_specification["is_pos1"] = [pirates_including_the_first]
# split_position_names["is_pos1"] = ['is_pos1']
split_position_specification["is_pos2"] = [pirates_including_the_first]
split_position_names["is_pos2"] = ['is_pos2']
split_position_specification["is_pos3"] = [pirates_including_the_first]
split_position_names["is_pos3"] = ['is_pos3']
split_position_specification["is_pos4"] = [pirates_including_the_first]
split_position_names["is_pos4"] = ['is_pos4']

split_position_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=split_position_specification,
model_type='MNL',
names=split_position_names)
split_position_mnl.fit_mle(np.zeros(24))
split_position_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.39 seconds.
Final log-likelihood: -18,138.9254


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,821
Method:                                MLE   Df Model:                           24
Date:                     Sun, 15 Jan 2023   Pseudo R-squ.:                   0.223
Time:                             01:43:33   Pseudo R-bar-squ.:               0.222
AIC:                            36,325.851   Log-Likelihood:            -18,138.925
BIC:                            36,511.414   LL-Null:                   -23,352.129
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
ASC_2_Sproggie        -1.1877      0.058    -20.528      0.000      -1.301      -1.074
ASC_3_Orvinn          -2.1886      0.068    -32.358      0.000      -2.321      -2.056
ASC_4_Lucky           -0.5932      0.055    -10.860      0.000      -0.700      -0.486
ASC_5_Edmund          -0.7330      0.055    -13.306      0.000      -0.841      -0.625
ASC_6_Peg Leg         -1.4415      0.065    -22.291      0.000      -1.568      -1.315
ASC_7_Bonnie          -1.1063      0.056    -19.592      0.000      -1.217      -0.996
ASC_8_Puffo           -1.6580      0.064    -25.933      0.000      -1.783      -1.533
ASC_9_Stuff           -2.4890      0.084    -29.677      0.000      -2.653      -2.325
ASC_10_Squire         -2.2447      0.081    -27.729      0.000      -2.403      -2.086
ASC_11_Crossblades    -2.0439      0.068    -30.086      0.000      -2.177      -1.911
ASC_12_Stripey        -1.2485      0.056    -22.226      0.000      -1.359      -1.138
ASC_13_Ned            -0.6826      0.056    -12.160      0.000      -0.793      -0.573
ASC_14_Fairfax        -1.4623      0.059    -24.919      0.000      -1.577      -1.347
ASC_15_Gooblah         0.9004      0.052     17.341      0.000       0.799       1.002
ASC_16_Franchisco     -0.5027      0.052     -9.697      0.000      -0.604      -0.401
ASC_17_Federismo      -0.5015      0.052     -9.669      0.000      -0.603      -0.400
ASC_18_Blackbeard     -1.2998      0.060    -21.691      0.000      -1.417      -1.182
ASC_19_Buck            0.4517      0.053      8.499      0.000       0.348       0.556
ASC_20_Tailhook       -0.4859      0.054     -8.943      0.000      -0.592      -0.379
PFA                    0.2135      0.009     24.925      0.000       0.197       0.230
NFA                    0.4186      0.012     34.926      0.000       0.395       0.442
is_pos2                0.1823      0.027      6.751      0.000       0.129       0.235
is_pos3                0.4633      0.026     17.850      0.000       0.412       0.514
is_pos4                0.7334      0.025     29.093      0.000       0.684       0.783
======================================================================================
"""

In [ ]:
# positional advantage is non-linear,
# so splitting it into 3 coefficients makes sense

In [169]:
backtest_model_with_cap(split_position_mnl, 1, history_rounds)

100%|██████████| 3369/3369 [03:25<00:00, 16.39it/s]


(33683, 33690, 83862, 50179, 2.489433066191748)

In [ ]:
# even though this is theoretically a better model than the previous one
# (it takes into account of non-linearity of positional advantage)
# the backtest ROI result is worse (dropped from 2.863 to 2.489)
# it probably takes an eternity for a model to converge to its true ROI
# given how long-tail the gambit bets are!

In [170]:
backtest_model_with_cap(split_position_mnl, 10000, history_rounds)

100%|██████████| 3369/3369 [03:10<00:00, 17.73it/s]


(326868956, 336900000, 659088561, 332219605, 1.9861074651231818)

In [8]:
# next we split FA coefficients for each pirate
per_pirate_fa_specification = OrderedDict()
per_pirate_fa_names = OrderedDict()
per_pirate_fa_specification["intercept"] = pirates_except_the_first
per_pirate_fa_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
per_pirate_fa_specification["pfa"] = pirates_including_the_first
per_pirate_fa_names["pfa"] = [f'PFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_specification["nfa"] = pirates_including_the_first
per_pirate_fa_names["nfa"] = [f'NFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_specification["is_pos1"] = [pirates_including_the_first]
per_pirate_fa_names["is_pos1"] = ['is_pos1']
# per_pirate_fa_specification["is_pos2"] = [pirates_including_the_first]
# per_pirate_fa_names["is_pos2"] = ['is_pos2']
per_pirate_fa_specification["is_pos3"] = [pirates_including_the_first]
per_pirate_fa_names["is_pos3"] = ['is_pos3']
per_pirate_fa_specification["is_pos4"] = [pirates_including_the_first]
per_pirate_fa_names["is_pos4"] = ['is_pos4']

per_pirate_fa_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=per_pirate_fa_specification,
model_type='MNL',
names=per_pirate_fa_names)
per_pirate_fa_mnl.fit_mle(np.zeros(62))
per_pirate_fa_mnl.get_statsmodels_summary()

Log-likelihood at zero: -24,197.7681
Initial Log-likelihood: -24,197.7681


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 6.90 seconds.
Final log-likelihood: -18,741.0438


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               17,455
Model:             Multinomial Logit Model   Df Residuals:                   17,393
Method:                                MLE   Df Model:                           62
Date:                     Fri, 05 May 2023   Pseudo R-squ.:                   0.226
Time:                             19:18:26   Pseudo R-bar-squ.:               0.223
AIC:                            37,606.088   Log-Likelihood:            -18,741.044
BIC:                            38,087.665   LL-Null:                   -24,197.768
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
ASC_2_Sproggie        -1.7828      0.169    -10.573      0.000      -2.113      -1.452
ASC_3_Orvinn          -2.7037      0.203    -13.345      0.000      -3.101      -2.307
ASC_4_Lucky           -0.6001      0.124     -4.840      0.000      -0.843      -0.357
ASC_5_Edmund          -1.1228      0.127     -8.864      0.000      -1.371      -0.875
ASC_6_Peg Leg         -1.7977      0.135    -13.292      0.000      -2.063      -1.533
ASC_7_Bonnie          -1.5412      0.142    -10.871      0.000      -1.819      -1.263
ASC_8_Puffo           -2.3532      0.203    -11.584      0.000      -2.751      -1.955
ASC_9_Stuff           -3.1495      0.188    -16.797      0.000      -3.517      -2.782
ASC_10_Squire         -2.5511      0.162    -15.761      0.000      -2.868      -2.234
ASC_11_Crossblades    -2.2096      0.166    -13.313      0.000      -2.535      -1.884
ASC_12_Stripey        -1.3936      0.143     -9.733      0.000      -1.674      -1.113
ASC_13_Ned            -0.8552      0.143     -5.988      0.000      -1.135      -0.575
ASC_14_Fairfax        -1.8561      0.156    -11.902      0.000      -2.162      -1.550
ASC_15_Gooblah         0.6265      0.125      5.011      0.000       0.381       0.872
ASC_16_Franchisco     -0.6487      0.132     -4.898      0.000      -0.908      -0.389
ASC_17_Federismo      -0.5343      0.132     -4.060      0.000      -0.792      -0.276
ASC_18_Blackbeard     -1.3695      0.148     -9.257      0.000      -1.660      -1.080
ASC_19_Buck            0.1587      0.123      1.288      0.198      -0.083       0.400
ASC_20_Tailhook       -0.8601      0.128     -6.739      0.000      -1.110      -0.610
PFA_1_Dan              0.1591      0.022      7.135      0.000       0.115       0.203
PFA_2_Sproggie         0.2643      0.035      7.580      0.000       0.196       0.333
PFA_3_Orvinn           0.2513      0.041      6.197      0.000       0.172       0.331
PFA_4_Lucky            0.1592      0.040      4.011      0.000       0.081       0.237
PFA_5_Edmund           0.2753      0.038      7.153      0.000       0.200       0.351
PFA_6_Peg Leg          0.3146      0.060      5.262      0.000       0.197       0.432
PFA_7_Bonnie           0.2460      0.036      6.813      0.000       0.175       0.317
PFA_8_Puffo            0.2855      0.039      7.301      0.000       0.209       0.362
PFA_9_Stuff            0.4086      0.076      5.382      0.000       0.260       0.557
PFA_10_Squire          0.1997      0.082      2.431      0.015       0.039       0.361
PFA_11_Crossblades     0.1740      0.046      3.768      0.000       0.083       0.265
PFA_12_Stripey         0.1971      0.033      5.999      0.000       0.133       0.262
PFA_13_Ned             0.2137      0.038      5.649      0.000       0.140       0.288
PFA_14_Fairfax         0.2453      0.034      7.113      0.000       0.178       0.313
PFA_15_Gooblah         0.2853      0.034      8.377      0.000  

In [191]:
backtest_model_with_cap(per_pirate_fa_mnl, 1, history_rounds)

100%|██████████| 3369/3369 [03:30<00:00, 16.00it/s]


(33684, 33690, 106976, 73292, 3.175482338972989)

In [192]:
backtest_model_with_cap(per_pirate_fa_mnl, 10000, history_rounds)

100%|██████████| 3369/3369 [03:13<00:00, 17.39it/s]


(326864356, 336900000, 696148633, 369284277, 2.096124300979519)

In [21]:
# next we split positional coefficients for each pirate
per_pirate_fa_and_pos_specification = OrderedDict()
per_pirate_fa_and_pos_names = OrderedDict()
# OK, for asthetic purposes and better-looking P-values
# (Dan is really close to Buck but everyone is much weaker than Gooblah,
#  that's why Buck has a bad-looking P-value) we exlude Gooblah from the intercepts
# You can interpret each exp(intercept) as "how many Goobs each pirate is equivalent to"
pirates_except_goob = [i for i in range(1, 21) if i != 15]
per_pirate_fa_and_pos_specification["intercept"] = pirates_except_goob
per_pirate_fa_and_pos_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_goob]
per_pirate_fa_and_pos_specification["pfa"] = pirates_including_the_first
per_pirate_fa_and_pos_names["pfa"] = [f'PFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_pos_specification["nfa"] = pirates_including_the_first
per_pirate_fa_and_pos_names["nfa"] = [f'NFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_pos_specification["position_factor"] = pirates_including_the_first
per_pirate_fa_and_pos_names["position_factor"] = [f'position_factor_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]

per_pirate_fa_and_pos_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=per_pirate_fa_and_pos_specification,
model_type='MNL',
names=per_pirate_fa_and_pos_names)
per_pirate_fa_and_pos_mnl.fit_mle(np.zeros(79))
per_pirate_fa_and_pos_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 5.37 seconds.
Final log-likelihood: -18,078.1295


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,766
Method:                                MLE   Df Model:                           79
Date:                     Sun, 15 Jan 2023   Pseudo R-squ.:                   0.226
Time:                             22:00:09   Pseudo R-bar-squ.:               0.222
AIC:                            36,314.259   Log-Likelihood:            -18,078.129
BIC:                            36,925.072   LL-Null:                   -23,352.129
==================================================================================================
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
ASC_1_Dan                         -0.6198      0.130     -4.763      0.000      -0.875      -0.365
ASC_2_Sproggie                    -2.3731      0.175    -13.545      0.000      -2.717      -2.030
ASC_3_Orvinn                      -3.3557      0.212    -15.861      0.000      -3.770      -2.941
ASC_4_Lucky                       -1.2663      0.130     -9.723      0.000      -1.522      -1.011
ASC_5_Edmund                      -1.7855      0.133    -13.471      0.000      -2.045      -1.526
ASC_6_Peg Leg                     -2.4166      0.143    -16.944      0.000      -2.696      -2.137
ASC_7_Bonnie                      -2.1906      0.150    -14.649      0.000      -2.484      -1.897
ASC_8_Puffo                       -2.9591      0.210    -14.098      0.000      -3.370      -2.548
ASC_9_Stuff                       -3.8389      0.204    -18.819      0.000      -4.239      -3.439
ASC_10_Squire                     -3.2848      0.180    -18.290      0.000      -3.637      -2.933
ASC_11_Crossblades                -2.8585      0.175    -16.306      0.000      -3.202      -2.515
ASC_12_Stripey                    -2.1179      0.149    -14.168      0.000      -2.411      -1.825
ASC_13_Ned                        -1.5461      0.148    -10.437      0.000      -1.836      -1.256
ASC_14_Fairfax                    -2.5004      0.163    -15.332      0.000      -2.820      -2.181
ASC_16_Franchisco                 -1.3108      0.139     -9.451      0.000      -1.583      -1.039
ASC_17_Federismo                  -1.1296      0.136     -8.288      0.000      -1.397      -0.862
ASC_18_Blackbeard                 -2.0534      0.154    -13.327      0.000      -2.355      -1.751
ASC_19_Buck                       -0.5117      0.129     -3.961      0.000      -0.765      -0.259
ASC_20_Tailhook                   -1.5502      0.132    -11.724      0.000      -1.809      -1.291
PFA_1_Dan                          0.1701      0.023      7.519      0.000       0.126       0.214
PFA_2_Sproggie                     0.2553      0.035      7.239      0.000       0.186       0.324
PFA_3_Orvinn                       0.2560      0.041      6.215      0.000       0.175       0.337
PFA_4_Lucky                        0.1616      0.041      3.971      0.000       0.082       0.241
PFA_5_Edmund                       0.2680      0.039      6.842      0.000       0.191       0.345
PFA_6_Peg Leg                      0.3154      0.061      5.195      0.000       0.196       0.434
PFA_7_Bonnie                       0.2449      0.037      6.654      0.000       0.173       0.317
PFA_8_Puffo                        0.2885      0.040      7.285      0.000       0.211       0.366
PFA_9_Stuff                        0.3963      0.078      5.085      0.000       0.244       0.549
PFA_10_Squire                      0.2337      0.083      2.802      0.005       0.070       0.397
PFA_11_Crossblades          

In [22]:
per_pirate_fa_and_per_pirate_pos_specification = OrderedDict()
per_pirate_fa_and_per_pirate_pos_names = OrderedDict()
pirates_except_goob = [i for i in range(1, 21) if i != 15]
per_pirate_fa_and_per_pirate_pos_specification["intercept"] = pirates_except_goob
per_pirate_fa_and_per_pirate_pos_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_goob]
per_pirate_fa_and_per_pirate_pos_specification["pfa"] = pirates_including_the_first
per_pirate_fa_and_per_pirate_pos_names["pfa"] = [f'PFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_per_pirate_pos_specification["nfa"] = pirates_including_the_first
per_pirate_fa_and_per_pirate_pos_names["nfa"] = [f'NFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_per_pirate_pos_specification["is_pos2"] = pirates_including_the_first
per_pirate_fa_and_per_pirate_pos_names["is_pos2"] = [f'is_pos2_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_per_pirate_pos_specification["is_pos3"] = pirates_including_the_first
per_pirate_fa_and_per_pirate_pos_names["is_pos3"] = [f'is_pos3_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_per_pirate_pos_specification["is_pos4"] = pirates_including_the_first
per_pirate_fa_and_per_pirate_pos_names["is_pos4"] = [f'is_pos4_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]

per_pirate_fa_and_per_pirate_pos_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=per_pirate_fa_and_per_pirate_pos_specification,
model_type='MNL',
names=per_pirate_fa_and_per_pirate_pos_names)
per_pirate_fa_and_per_pirate_pos_mnl.fit_mle(np.zeros(119))
per_pirate_fa_and_per_pirate_pos_mnl.get_statsmodels_summary()

Log-likelihood at zero: -24,197.7681
Initial Log-likelihood: -24,197.7681


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 9.02 seconds.
Final log-likelihood: -18,713.2221


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               17,455
Model:             Multinomial Logit Model   Df Residuals:                   17,336
Method:                                MLE   Df Model:                          119
Date:                     Sat, 06 May 2023   Pseudo R-squ.:                   0.227
Time:                             13:45:57   Pseudo R-bar-squ.:               0.222
AIC:                            37,664.444   Log-Likelihood:            -18,713.222
BIC:                            38,588.763   LL-Null:                   -24,197.768
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
ASC_1_Dan                 -0.5506      0.155     -3.541      0.000      -0.855      -0.246
ASC_2_Sproggie            -2.3848      0.203    -11.771      0.000      -2.782      -1.988
ASC_3_Orvinn              -3.4786      0.244    -14.251      0.000      -3.957      -3.000
ASC_4_Lucky               -1.3890      0.160     -8.670      0.000      -1.703      -1.075
ASC_5_Edmund              -1.9177      0.163    -11.775      0.000      -2.237      -1.598
ASC_6_Peg Leg             -2.5675      0.185    -13.894      0.000      -2.930      -2.205
ASC_7_Bonnie              -2.3143      0.180    -12.861      0.000      -2.667      -1.962
ASC_8_Puffo               -2.8314      0.234    -12.089      0.000      -3.290      -2.372
ASC_9_Stuff               -3.9019      0.259    -15.056      0.000      -4.410      -3.394
ASC_10_Squire             -3.5882      0.256    -14.006      0.000      -4.090      -3.086
ASC_11_Crossblades        -3.1482      0.225    -13.997      0.000      -3.589      -2.707
ASC_12_Stripey            -2.1693      0.181    -12.010      0.000      -2.523      -1.815
ASC_13_Ned                -1.7063      0.176     -9.672      0.000      -2.052      -1.361
ASC_14_Fairfax            -2.5503      0.191    -13.328      0.000      -2.925      -2.175
ASC_16_Franchisco         -1.2579      0.163     -7.725      0.000      -1.577      -0.939
ASC_17_Federismo          -1.0598      0.159     -6.678      0.000      -1.371      -0.749
ASC_18_Blackbeard         -2.1826      0.189    -11.568      0.000      -2.552      -1.813
ASC_19_Buck               -0.5606      0.152     -3.691      0.000      -0.858      -0.263
ASC_20_Tailhook           -1.6608      0.159    -10.417      0.000      -1.973      -1.348
PFA_1_Dan                  0.1575      0.022      7.085      0.000       0.114       0.201
PFA_2_Sproggie             0.2631      0.035      7.541      0.000       0.195       0.331
PFA_3_Orvinn               0.2510      0.041      6.181      0.000       0.171       0.331
PFA_4_Lucky                0.1596      0.040      3.999      0.000       0.081       0.238
PFA_5_Edmund               0.2765      0.039      7.163      0.000       0.201       0.352
PFA_6_Peg Leg              0.3169      0.060      5.291      0.000       0.199       0.434
PFA_7_Bonnie               0.2477      0.036      6.848      0.000       0.177       0.319
PFA_8_Puffo                0.2858      0.039      7.319      0.000       0.209       0.362
PFA_9_Stuff                0.4114      0.076      5.405      0.000       0.262       0.561
PFA_10_Squire              0.1973      0.082      2.396      0.017       0.036       0.359
PFA_11_Crossblades         0.1735      0.046      3.750      0.000       0.083       0.264
PFA_12_Stripey             0.1990      0.033      6.019      0.000       0.134       0.264
PFA_13_Ned                 0.2165      0.038      5.686      0.000       0.142       0.291
PFA_14_Fair

In [ ]:
per_pirate_fa_and_pos_specification = OrderedDict()
per_pirate_fa_and_pos_names = OrderedDict()
# OK, for asthetic purposes and better-looking P-values
# (Dan is really close to Buck but everyone is much weaker than Gooblah,
#  that's why Buck has a bad-looking P-value) we exlude Gooblah from the intercepts
# You can interpret each exp(intercept) as "how many Goobs each pirate is equivalent to"
pirates_except_goob = [i for i in range(1, 21) if i != 15]
per_pirate_fa_and_pos_specification["intercept"] = pirates_except_goob
per_pirate_fa_and_pos_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_goob]
per_pirate_fa_and_pos_specification["pfa"] = pirates_including_the_first
per_pirate_fa_and_pos_names["pfa"] = [f'PFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_pos_specification["nfa"] = pirates_including_the_first
per_pirate_fa_and_pos_names["nfa"] = [f'NFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_pos_specification["position_factor"] = pirates_including_the_first
per_pirate_fa_and_pos_names["position_factor"] = [f'position_factor_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]

per_pirate_fa_and_pos_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=per_pirate_fa_and_pos_specification,
model_type='MNL',
names=per_pirate_fa_and_pos_names)
per_pirate_fa_and_pos_mnl.fit_mle(np.zeros(79))
per_pirate_fa_and_pos_mnl.get_statsmodels_summary()

In [400]:
# next we split opening odds factor for each pirate
# model that incorporates opening odds tend to overfit and has very bad ROI in cross validation
pirates_except_goob = [i for i in range(1, 21) if i != 15]
per_pirate_fa_and_pos_and_loiw_specification = OrderedDict()
per_pirate_fa_and_pos_and_loiw_names = OrderedDict()
per_pirate_fa_and_pos_and_loiw_specification["intercept"] = pirates_except_goob
per_pirate_fa_and_pos_and_loiw_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_goob]
per_pirate_fa_and_pos_and_loiw_specification["pfa"] = pirates_including_the_first
per_pirate_fa_and_pos_and_loiw_names["pfa"] = [f'PFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_pos_and_loiw_specification["nfa"] = pirates_including_the_first
per_pirate_fa_and_pos_and_loiw_names["nfa"] = [f'NFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_pos_and_loiw_specification["position_factor"] = pirates_including_the_first
per_pirate_fa_and_pos_and_loiw_names["position_factor"] = [f'position_factor_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_pos_and_loiw_specification["log_opening_implied_winrate"] = pirates_including_the_first
per_pirate_fa_and_pos_and_loiw_names["log_opening_implied_winrate"] = [f'loiw_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]

per_pirate_fa_and_pos_and_loiw_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=per_pirate_fa_and_pos_and_loiw_specification,
model_type='MNL',
names=per_pirate_fa_and_pos_and_loiw_names)
per_pirate_fa_and_pos_and_loiw_mnl.fit_mle(np.zeros(99))
per_pirate_fa_and_pos_and_loiw_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 8.58 seconds.
Final log-likelihood: -18,042.9103


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,746
Method:                                MLE   Df Model:                           99
Date:                     Fri, 06 Jan 2023   Pseudo R-squ.:                   0.227
Time:                             16:42:33   Pseudo R-bar-squ.:               0.223
AIC:                            36,283.821   Log-Likelihood:            -18,042.910
BIC:                            37,049.270   LL-Null:                   -23,352.129
==================================================================================================
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
ASC_1_Dan                         -0.3213      0.329     -0.976      0.329      -0.966       0.324
ASC_2_Sproggie                    -1.7361      0.372     -4.670      0.000      -2.465      -1.007
ASC_3_Orvinn                      -2.3232      0.480     -4.843      0.000      -3.263      -1.383
ASC_4_Lucky                       -0.5717      0.330     -1.734      0.083      -1.218       0.074
ASC_5_Edmund                      -1.1790      0.337     -3.493      0.000      -1.840      -0.518
ASC_6_Peg Leg                     -1.5394      0.376     -4.093      0.000      -2.277      -0.802
ASC_7_Bonnie                      -1.6900      0.353     -4.786      0.000      -2.382      -0.998
ASC_8_Puffo                       -2.7058      0.448     -6.040      0.000      -3.584      -1.828
ASC_9_Stuff                       -2.3778      0.617     -3.856      0.000      -3.587      -1.169
ASC_10_Squire                     -2.0556      0.530     -3.881      0.000      -3.094      -1.018
ASC_11_Crossblades                -1.6387      0.456     -3.597      0.000      -2.532      -0.746
ASC_12_Stripey                    -1.1021      0.361     -3.055      0.002      -1.809      -0.395
ASC_13_Ned                        -0.8256      0.345     -2.394      0.017      -1.502      -0.150
ASC_14_Fairfax                    -1.6615      0.385     -4.319      0.000      -2.415      -0.908
ASC_16_Franchisco                 -0.8589      0.331     -2.596      0.009      -1.508      -0.210
ASC_17_Federismo                  -0.5870      0.332     -1.766      0.077      -1.238       0.064
ASC_18_Blackbeard                 -1.2877      0.366     -3.521      0.000      -2.005      -0.571
ASC_19_Buck                       -0.2524      0.320     -0.789      0.430      -0.880       0.375
ASC_20_Tailhook                   -1.0454      0.330     -3.165      0.002      -1.693      -0.398
PFA_1_Dan                          0.1350      0.023      5.796      0.000       0.089       0.181
PFA_2_Sproggie                     0.1984      0.038      5.204      0.000       0.124       0.273
PFA_3_Orvinn                       0.2025      0.045      4.520      0.000       0.115       0.290
PFA_4_Lucky                        0.0884      0.043      2.055      0.040       0.004       0.173
PFA_5_Edmund                       0.2004      0.042      4.733      0.000       0.117       0.283
PFA_6_Peg Leg                      0.2382      0.063      3.755      0.000       0.114       0.363
PFA_7_Bonnie                       0.1998      0.040      5.004      0.000       0.122       0.278
PFA_8_Puffo                        0.2769      0.044      6.258      0.000       0.190       0.364
PFA_9_Stuff                        0.3310      0.082      4.057      0.000       0.171       0.491
PFA_10_Squire                      0.1802      0.087      2.082      0.037       0.011       0.350
PFA_11_Crossblades          

In [23]:
backtest_model_with_cap(per_pirate_fa_and_per_pirate_pos_mnl, 1, history_rounds)

100%|██████████| 3491/3491 [03:40<00:00, 15.83it/s]


(34910, 34910, 103298, 68388, 2.9589802348897165)

In [398]:
backtest_model_with_cap(per_pirate_fa_and_pos_and_loiw_mnl, 1, history_rounds)

100%|██████████| 3369/3369 [03:28<00:00, 16.14it/s]


(33648, 33690, 97298, 63650, 2.8892846542000594)

In [203]:
backtest_model_with_cap(per_pirate_fa_and_pos_mnl, 1, history_rounds)

100%|██████████| 3369/3369 [03:39<00:00, 15.38it/s]


(33690, 33690, 98598, 64908, 2.9266251113089936)

In [204]:
backtest_model_with_cap(per_pirate_fa_and_pos_mnl, 10, history_rounds)

100%|██████████| 3369/3369 [03:36<00:00, 15.54it/s]


(336883, 336900, 985980, 649097, 2.926675571386168)

In [205]:
backtest_model_with_cap(per_pirate_fa_and_pos_mnl, 100, history_rounds)

100%|██████████| 3369/3369 [03:35<00:00, 15.63it/s]


(3364338, 3369000, 10426400, 7062062, 3.096189373701395)

In [206]:
backtest_model_with_cap(per_pirate_fa_and_pos_mnl, 1000, history_rounds)

100%|██████████| 3369/3369 [03:27<00:00, 16.20it/s]


(33424426, 33690000, 88346548, 54922122, 2.6302203027604634)

In [207]:

backtest_model_with_cap(per_pirate_fa_and_pos_mnl, 10000, history_rounds)

100%|██████████| 3369/3369 [03:20<00:00, 16.84it/s]


(326901918, 336900000, 741728401, 414826483, 2.2313044909468687)

In [215]:
backtest_model_with_cap(per_pirate_fa_and_pos_mnl, 15000, history_rounds)

100%|██████████| 3369/3369 [03:23<00:00, 16.56it/s]


(485506397, 505350000, 1053954532, 568448135, 2.1248602651627584)

In [22]:
def get_current_round_df_from_file(path):
    columns = ['round', 'arena', 'pirate1', 'pirate2', 'pirate3', 'pirate4', 'fa1', 'fa2', 'fa3', 'fa4', 'pfa1', 'pfa2', 'pfa3', 'pfa4', 'nfa1', 'nfa2', 'nfa3', 'nfa4', 'opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4', 'closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']
    df = pd.DataFrame([], columns=columns)
    with open(path) as json_file:
        data = json.load(json_file)
        round = data["round"]
        for arena in range(5):
            pirates = data["pirates"][arena]
            # older rounds don't have fa data
            fas = [0 for pirate in pirates]
            positive_fas = [0 for pirate in pirates]
            negative_fas = [0 for pirate in pirates]
            if "foods" in data:
                foods = data["foods"][arena]
                positive_fas = [sum(POSITIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                negative_fas = [-sum(NEGATIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                fas = [sum(x) for x in zip(positive_fas, negative_fas)]
            pirate1, pirate2, pirate3, pirate4 = pirates
            fa1, fa2, fa3, fa4 = fas
            pfa1, pfa2, pfa3, pfa4 = positive_fas
            nfa1, nfa2, nfa3, nfa4 = negative_fas
            _, opening_odds1, opening_odds2, opening_odds3, opening_odds4 = data["openingOdds"][arena]
            _, closing_odds1, closing_odds2, closing_odds3, closing_odds4 = data["currentOdds"][arena]
            df_temp = pd.DataFrame([[round, arena, pirate1, pirate2, pirate3, pirate4, fa1, fa2, fa3, fa4, pfa1, pfa2, pfa3, pfa4, nfa1, nfa2, nfa3, nfa4, opening_odds1, opening_odds2, opening_odds3, opening_odds4, closing_odds1, closing_odds2, closing_odds3, closing_odds4]], columns=columns)
            df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [23]:
def get_round(round_number):
    return convert_to_long_format_without_win(convert_df(get_current_round_df_from_file(f"raw_json/{round_number}.json")))
round_8643 = get_round(8643)
round_8643

,round,arena,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,8643,0,2,-1,2,-3,1,-0.1821,1,0,0,0,13,13,43215,-2.564949
1,8643,0,16,2,2,0,2,0.0000,0,1,0,0,2,2,43215,-0.693147
2,8643,0,3,0,1,-1,3,0.2814,0,0,1,0,13,13,43215,-2.564949
3,8643,0,5,2,2,0,4,0.5523,0,0,0,1,2,2,43215,-0.693147
4,8643,1,9,-1,0,-1,1,-0.1821,1,0,0,0,13,13,43216,-2.564949
5,8643,1,18,1,2,-1,2,0.0000,0,1,0,0,13,13,43216,-2.564949
6,8643,1,1,4,4,0,3,0.2814,0,0,1,0,2,2,43216,-0.693147
7,8643,1,8,-1,2,-3,4,0.5523,0,0,0,1,13,13,43216,-2.564949
8,8643,2,19,0,1,-1,1,-0.1821,1,0,0,0,2,2,43217,-0.693147
9,8643,2,13,2,3,-1,2,0.0000,0,1,0,0,2,2,43217,-0.693147


In [24]:
per_pirate_fa_and_pos_mnl.predict(round_8643)

array([0.05343339, 0.38704053, 0.04310275, 0.51642334, 0.0095265 ,
       0.07873558, 0.8718608 , 0.03987712, 0.50995024, 0.29752494,
       0.05561974, 0.13690508, 0.08281387, 0.3355578 , 0.40114837,
       0.18047996, 0.04304779, 0.86484364, 0.05053249, 0.04157608])

In [278]:
# do 5-fold cross-validation of data on long_format
# split data based on "round"
from sklearn.model_selection import GroupShuffleSplit
def cross_validate_model(specification, names, cap, long_format, random_state=None):
    kf = GroupShuffleSplit(n_splits=5, random_state=random_state)
    ROIs = []
    for train_index, test_index in kf.split(long_format, groups=long_format["round"]):
        train = long_format.iloc[train_index]
        test = long_format.iloc[test_index]
        trained_mnl = pl.create_choice_model(data=train,
            alt_id_col='pirate',
            obs_id_col='match_id',
            choice_col='win',
            specification=specification,
            model_type='MNL',
            names=names)
        trained_mnl.fit_mle(np.zeros(sum(len(v) for v in specification.values())))
        result = backtest_model_with_cap(trained_mnl, cap, test["round"].unique())
        print(result)
        ROIs.append(result[-1])
    return ROIs, np.mean(ROIs)

In [403]:
cross_validate_model(per_pirate_fa_and_pos_specification, per_pirate_fa_and_pos_names, 1, long_format)

Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 4.70 seconds.
Final log-likelihood: -14,405.6047


100%|██████████| 674/674 [00:42<00:00, 15.70it/s]

(6740, 6740, 7482, 742, 1.1100890207715133)
Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


...


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 6.44 seconds.
Final log-likelihood: -14,429.3325


100%|██████████| 674/674 [00:42<00:00, 15.99it/s]

(6740, 6740, 36762, 30022, 5.454302670623146)


([1.1100890207715133,
  2.518694362017804,
  2.0379821958456974,
  3.9902077151335313,
  5.454302670623146],
 3.022255192878338)

In [252]:
cross_validate_model(per_pirate_fa_and_pos_specification, per_pirate_fa_and_pos_names, 10, long_format)

Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 6.84 seconds.
Final log-likelihood: -14,418.7432


100%|██████████| 674/674 [00:42<00:00, 15.75it/s]

(67396, 67400, 112900, 45504, 1.67513353115727)
Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


...


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 4.46 seconds.
Final log-likelihood: -14,472.5148


100%|██████████| 674/674 [00:43<00:00, 15.55it/s]

(67398, 67400, 163310, 95912, 2.423026706231454)


([1.67513353115727,
  2.119881305637982,
  3.947507418397626,
  1.7915281899109792,
  2.423026706231454],
 2.3914154302670623)

In [253]:
cross_validate_model(per_pirate_fa_and_pos_specification, per_pirate_fa_and_pos_names, 100, long_format)

Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 5.12 seconds.
Final log-likelihood: -14,434.3796


100%|██████████| 674/674 [00:42<00:00, 15.78it/s]


(672945, 674000, 2831000, 2158055, 4.201862017804155)
Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


...

/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 4.92 seconds.
Final log-likelihood: -14,455.5643


100%|██████████| 674/674 [00:43<00:00, 15.59it/s]

(672971, 674000, 628900, -44071, 0.9346127596439169)


([4.201862017804155,
  1.9088679525222552,
  1.5818813056379821,
  3.9318442136498515,
  0.9346127596439169],
 2.511813649851632)

In [254]:
cross_validate_model(per_pirate_fa_and_pos_specification, per_pirate_fa_and_pos_names, 1000, long_format)

Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 6.97 seconds.
Final log-likelihood: -14,515.0251


100%|██████████| 674/674 [00:42<00:00, 15.80it/s]

(6682166, 6740000, 17845756, 11163590, 2.6563189910979226)
Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


...

/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 4.75 seconds.
Final log-likelihood: -14,480.4833


100%|██████████| 674/674 [00:43<00:00, 15.67it/s]

(6680222, 6740000, 17468936, 10788714, 2.60069940652819)


([2.6563189910979226,
  2.9048704747774483,
  2.5710675074183977,
  2.928624629080119,
  2.60069940652819],
 2.7323162017804155)

In [255]:
cross_validate_model(per_pirate_fa_and_pos_specification, per_pirate_fa_and_pos_names, 10000, long_format)

Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 6.47 seconds.
Final log-likelihood: -14,409.7056


100%|██████████| 674/674 [00:39<00:00, 17.24it/s]


(65389429, 67400000, 131089812, 65700383, 1.9747831305637982)
Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


...


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 5.01 seconds.
Final log-likelihood: -14,497.6654


100%|██████████| 674/674 [00:40<00:00, 16.68it/s]

(65237724, 67400000, 148489675, 83251951, 2.235192151335312)


([1.9747831305637982,
  2.0548693620178042,
  2.009290341246291,
  2.0185381602373886,
  2.235192151335312],
 2.0585346290801185)

In [256]:
cross_validate_model(per_pirate_fa_and_pos_specification, per_pirate_fa_and_pos_names, 3162, long_format)

Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 6.27 seconds.
Final log-likelihood: -14,465.0890


100%|██████████| 674/674 [00:40<00:00, 16.62it/s]

(20998059, 21311880, 48451012, 27452953, 2.2881525703035113)
Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


...


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 6.82 seconds.
Final log-likelihood: -14,431.1700


100%|██████████| 674/674 [00:39<00:00, 17.01it/s]

(20969660, 21311880, 48127966, 27158306, 2.2743270889288043)


([2.2881525703035113,
  2.096904168003949,
  2.001549605196726,
  1.9781698282835678,
  2.2743270889288043],
 2.127820652143311)

In [399]:
# model that incorporates opening odds tend to overfit and has very bad ROI in cross validation
cross_validate_model(per_pirate_fa_and_pos_and_loiw_specification, per_pirate_fa_and_pos_and_loiw_names, 1, long_format)

Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 9.31 seconds.
Final log-likelihood: -14,424.2084


100%|██████████| 674/674 [00:41<00:00, 16.30it/s]


(6740, 6740, 9707, 2967, 1.440207715133531)
Log-likelihood at zero: -18,680.3165
Initial Log-likelihood: -18,680.3165


...


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 6.82 seconds.
Final log-likelihood: -14,471.7902


100%|██████████| 674/674 [00:41<00:00, 16.18it/s]

(6740, 6740, 21773, 15033, 3.2304154302670622)


([1.440207715133531,
  2.1002967359050446,
  1.690059347181009,
  1.4494065281899111,
  3.2304154302670622],
 1.9820771513353115)

In [244]:
# reimplementation in python of our heirloom snakeoil model for probabilities
from fractions import Fraction

class SnakeOilModel:
    def __init__(self):
        pass
    def predict(self, df):
        # some conversion
        opening_odds_series = df["opening_odds"]
        opening_odds = np.ones((5, 5), dtype=Fraction)
        for i in range(5):
            for j in range(1, 5):
                opening_odds[i][j] = opening_odds_series[i*4 + j - 1]
        # below courtesy of nsheng
        min_probs = np.zeros([5, 5], dtype=Fraction)
        max_probs = np.zeros([5, 5], dtype=Fraction)
        std_probs = np.zeros([5, 5], dtype=Fraction)

        for a in range(5):
            for p in range(1, 5):
                if opening_odds[a, p] == 13:
                    min_probs[a, p] = 0
                    max_probs[a, p] = Fraction(1, 13)
                elif opening_odds[a, p] == 2:
                    min_probs[a, p] = Fraction(1, 3)
                    max_probs[a, p] = 1
                else:
                    min_probs[a, p] = Fraction(1, opening_odds[a, p] + 1)
                    max_probs[a, p] = Fraction(1, opening_odds[a, p])
            total_min = np.sum(min_probs[a])
            total_max = np.sum(max_probs[a])

            for p in range(1, 5):
                min_probs[a, p] = max(min_probs[a, p], 1 + max_probs[a, p] - total_max)
                max_probs[a, p] = min(max_probs[a, p], 1 + min_probs[a, p] - total_min)
                if opening_odds[a, p] == 13:
                    std_probs[a, p] = Fraction(1, 20)
                else:
                    std_probs[a, p] = Fraction(min_probs[a, p] + max_probs[a, p], 2)

            for odd in range(2, 14):
                total_std = np.sum(std_probs[a])
                if total_std == 1:
                    break
                count_smaller = np.count_nonzero(opening_odds[a, 1:] <= odd)
                if count_smaller == 0:
                    continue
                std_to_min = np.sum(
                    np.where(
                        opening_odds[a] <= odd,
                        std_probs[a] - min_probs[a],
                        np.zeros([5], dtype=Fraction),
                    )
                )
                smallest_range = np.min(
                    np.where(
                        opening_odds[a, 1:] <= odd,
                        max_probs[a, 1:] - min_probs[a, 1:],
                        np.ones([4], dtype=Fraction)
                    )
                )
                if (
                    total_std - std_to_min <= 1
                    and std_to_min + 1 - total_std <= smallest_range * count_smaller
                ):
                    remaining_gap = Fraction(std_to_min + 1 - total_std, count_smaller)
                    for p in range(1, 5):
                        if opening_odds[a, p] <= odd:
                            std_probs[a, p] = min_probs[a, p] + remaining_gap
                    break

        std_probs[:, 0] = np.ones([5])

        # convert things back
        return np.reshape(std_probs[:, 1:], (20)).astype(np.float64)

SnakeOilModel().predict(tmp)


array([0.225     , 0.05      , 0.05      , 0.675     , 0.05      ,
       0.11805556, 0.05      , 0.78194444, 0.43690476, 0.1547619 ,
       0.18333333, 0.225     , 0.05      , 0.225     , 0.54166667,
       0.18333333, 0.08712121, 0.30429293, 0.30429293, 0.30429293])

In [245]:
backtest_model_with_cap(SnakeOilModel(), 1, history_rounds)

100%|██████████| 3369/3369 [03:03<00:00, 18.31it/s]


(33389, 33690, 72097, 38708, 2.148946274859009)

In [246]:
backtest_model_with_cap(SnakeOilModel(), 10000, history_rounds)

100%|██████████| 3369/3369 [02:51<00:00, 19.60it/s]


(330628247, 336900000, 687559223, 356930976, 2.0594567408726627)

In [257]:
tmp

,round,arena,win,pirate,fa,position,opening_odds,closing_odds,match_id
0,8641,0,0,19,2,1,4,4,43205
1,8641,0,0,12,4,2,13,13,43205
2,8641,0,0,2,3,3,13,13,43205
3,8641,0,1,15,3,4,2,2,43205
4,8641,1,0,10,1,1,13,13,43206
5,8641,1,0,20,-1,2,8,12,43206
6,8641,1,0,8,-2,3,13,13,43206
7,8641,1,1,4,3,4,2,2,43206
8,8641,2,1,17,4,1,2,2,43207
9,8641,2,0,7,-1,2,6,6,43207


In [25]:
import string
def parse_bets(s):
    indices = [string.ascii_lowercase.find(c) for c in s]
    values = []
    for index in indices:
        values.append(index // 5)
        values.append(index % 5)

In [26]:
def get_best_10_bets(round, odds, model, cap):
    return sorted(list(get_profitable_bets(
        odds,
        get_estimated_probabilities(get_round(round), model), cap)),
        key=lambda x: x[3], reverse=True)[:10]

In [27]:
def get_bet_url(round, bets):
    bets_array = [v for bet in bets for v in bet[0]]
    # turn bets_array into chunks of 2 (pad 0 if necessary)
    if len(bets_array) % 2 == 1:
        bets_array.append(0)
    bets_chunks = [bets_array[i:i+2] for i in range(0, len(bets_array), 2)]
    # turn bets_chunks into a string
    bets_string = "".join([string.ascii_lowercase[5 * x + y] for x, y in bets_chunks])
    return f"/#round={round}&b={bets_string}"

In [28]:
from coloraide import Color, stop
def get_background_for_payout(payout):
    return Color.interpolate(['#50c17f', stop('transparent', np.exp(-1)), '#f76c6c'], space='srgb', premultiplied=True)(np.exp(-payout)).to_string()

get_background_for_payout(1.2)

'rgb(80 193 127 / 0.18127)'

In [29]:
def get_payouts(round, model, odds_override=None):
    round_df = get_round(round)
    estimated_probabilities = get_estimated_probabilities(round_df, model)
    estimated_probabilities_series = pd.Series(estimated_probabilities, name="estimated_probabilities")
    round_df = round_df.join(estimated_probabilities_series)
    if odds_override is not None:
        round_df["custom_odds"] = pd.Series(odds_override, name="custom_odds")
    else:
        round_df["custom_odds"] = round_df["closing_odds"]
    round_df["payouts"] = round_df.apply(lambda row: row["custom_odds"] * row["estimated_probabilities"], axis=1)
    return round_df.style.applymap(lambda x: f"background-color: {get_background_for_payout(x)}", subset=["payouts"])

In [48]:
current_round = 8655

In [49]:
get_round(current_round)['closing_odds'].tolist()

[13, 13, 2, 2, 3, 13, 2, 6, 3, 10, 3, 3, 13, 10, 2, 13, 13, 5, 2, 3]

In [50]:
per_pirate_fa_and_pos_mnl.predict(get_round(current_round))

array([0.08296103, 0.05426712, 0.36066594, 0.50210591, 0.25098267,
       0.04119835, 0.57064248, 0.1371765 , 0.29310417, 0.06049772,
       0.31887228, 0.32752583, 0.05752514, 0.11706066, 0.77035466,
       0.05505953, 0.01352436, 0.27610916, 0.41918506, 0.29118142])

In [51]:
manual_odds = [13, 13, 2, 2, 3, 13, 2, 6, 3, 10, 3, 3, 13, 10, 2, 13, 13, 5, 2, 3]

In [52]:
get_payouts(current_round, per_pirate_fa_and_pos_mnl, odds_override=manual_odds)

,round,arena,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate,intercept,estimated_probabilities,custom_odds,payouts
0,8655,0,7,-1,2,-3,1,-0.182100,1,0,0,0,13,13,43275,-2.564949,1.000000,0.082961,13,1.078493
1,8655,0,3,1,1,0,2,0.000000,0,1,0,0,13,13,43275,-2.564949,1.000000,0.054267,13,0.705473
2,8655,0,5,1,1,0,3,0.281400,0,0,1,0,2,2,43275,-0.693147,1.000000,0.360666,2,0.721332
3,8655,0,13,1,2,-1,4,0.552300,0,0,0,1,2,2,43275,-0.693147,1.000000,0.502106,2,1.004212
4,8655,1,19,-1,0,-1,1,-0.182100,1,0,0,0,3,3,43276,-1.098612,1.000000,0.250983,3,0.752948
5,8655,1,18,0,3,-3,2,0.000000,0,1,0,0,13,13,43276,-2.564949,1.000000,0.041198,13,0.535579
6,8655,1,1,1,1,0,3,0.281400,0,0,1,0,2,2,43276,-0.693147,1.000000,0.570642,2,1.141285
7,8655,1,20,-2,1,-3,4,0.552300,0,0,0,1,6,6,43276,-1.791759,1.000000,0.137176,6,0.823059
8,8655,2,17,1,2,-1,1,-0.182100,1,0,0,0,3,3,43277,-1.098612,1.000000,0.293104,3,0.879313
9,8655,2,8,0,3,-3,2,0.000000,0,1,0,0,12,10,43277,-2.484907,1.000000,0.060498,10,0.604977


In [55]:
get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_pos_mnl, 1)

[((1, 3, 0, 3, 2), 1, 2.618081183866026, 1.618081183866026, 260),
 ((1, 3, 4, 3, 2), 1, 2.572467666089591, 1.572467666089591, 780),
 ((1, 3, 3, 3, 2), 1, 2.504500545404314, 1.5045005454043139, 780),
 ((4, 3, 0, 3, 2), 1, 2.437760063461702, 1.437760063461702, 40),
 ((0, 3, 0, 3, 2), 1, 2.42753570939919, 1.4275357093991898, 20),
 ((4, 3, 4, 3, 2), 1, 2.395288190291903, 1.395288190291903, 120),
 ((0, 3, 4, 3, 2), 1, 2.3852419700316037, 1.3852419700316037, 60),
 ((4, 3, 3, 3, 2), 1, 2.33200232526369, 1.33200232526369, 120),
 ((0, 3, 3, 3, 2), 1, 2.322221536003307, 1.322221536003307, 60),
 ((1, 3, 1, 3, 2), 1, 2.3021115222911384, 1.3021115222911384, 780)]

In [56]:
get_bet_url(current_round, get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_pos_mnl, 1))

'/#round=8655&b=idltrisoprddotrdxosrdslqr'

In [38]:
get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_pos_mnl, 11404)

[((0, 1, 4, 3, 0), 10417, 32922.92071955713, 22505.92071955713, 1000000),
 ((4, 1, 0, 3, 0), 11400, 33052.303065134416, 21652.303065134416, 820800),
 ((4, 0, 4, 3, 0), 11400, 30356.05371323205, 18956.05371323205, 820800),
 ((4, 1, 4, 4, 0), 10417, 28259.970093812564, 17842.970093812564, 1000000),
 ((3, 1, 0, 3, 0), 11400, 28052.731117142604, 16652.731117142604, 820800),
 ((0, 1, 0, 3, 0), 11400, 27776.889494742412, 16376.889494742412, 273600),
 ((0, 1, 4, 4, 0), 11400, 25991.399649147683, 14591.399649147683, 364800),
 ((3, 0, 4, 3, 0), 11400, 25764.32301606012, 14364.32301606012, 820800),
 ((0, 0, 4, 3, 0), 11400, 25510.983238513476, 14110.983238513476, 273600),
 ((0, 1, 0, 3, 4), 11400, 25180.433352611224, 13780.433352611224, 547200)]

In [54]:
get_bet_url(current_round, get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_pos_mnl, 11404))

'/#round=8655&b=xdkprdxksrudkqradoeraxodr'